# 1D model for ice formation from INP:  ABIFM example
## Assumptions
— cloud conditions are assumed steady-state (unaffected by weak ice formation)  
— all aerosol are assumed to be activated in updrafts and restored in downdrafts  
— thus, all in-cloud aerosol (including INP) are within a droplet suitable for immersion freezing  
## Step-by-step
— profiles of temperature, RH, and cloud water mixing ratio are read in from LES output  
— model predicts evolution of Ninp(z,t) and Nice(z,t)  
— precipitation rates are estimated from LES number-weighted value at cloud base  
— predicted nucleation rate profiles are also saved and plotted  
## In this example
— assume a monodisperse INP population with fixed size and ABIFM parameters  
— INP profile is initialized to uniform number concentration and ABIFM parameters specified  
— optional assumption that entrained aerosol are initially interstitial, using differing "run the model" cells  

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# read in LES data

les_file = 'data_les/shi3_isdac_sfc6_pTqv_fthqv_lw_3_abifm_final/dharma.soundings.cdf'
les = xr.open_dataset(les_file)
les = les.sel(time=3600*3,method='nearest')       # harvest output after LES is steady (3h)
izt_pbl_top = np.max(np.where(les.ql.data>1.e-6)) # identify PBL top z-index
les = les[dict(zt=np.arange(izt_pbl_top+1))]      # 1D model domain extends only to PBL top
les = les[['T','ql','RH','pflux_3','ntot_3']]     # retain only variables needed

# rename vertical grid for plotting and make some unit conversions

les = les.rename({'zt':'height'})
les['height'].attrs['long_name'] = 'Height'
les['height'].attrs['units'] = 'm'
les.RH.data = les.RH.data / 100.
les.RH.attrs['units'] = '-'
les.ql.data = les.ql.data * 1000.
les.ql.attrs['units'] = 'g kg-1'
les = les.rename({'pflux_3':'prec'})
les = les.rename({'ntot_3':'Ni'})
les.Ni.data = les.Ni.data * 1000.
les.Ni.attrs['units'] = 'L-1'

# calculate number-weighted precip rate at cloud base

les = les.assign( P_Ni = les['prec']/les['Ni'] )
les['P_Ni'].attrs['units'] = 'mm h-1 / L-1'
izt_cloud_base = np.max(np.where(les.ql.data<1.e-3)) # identify cloud base - IS NOTE 4/28/21: this finds the layer just below cloud base meaning that precip rate suffesrs from sublimation (will be correct in .py files)
Pcb_per_Ni = les.P_Ni[izt_cloud_base] # [mm h-1 / L-1]

# calculate ∆aw for ABIFM

les = les.assign( delta_aw = les['RH'] -
    ( np.exp(9.550426-5723.265/les['T']+3.53068*np.log(les['T'])-0.00728332*les['T']) /
    ( np.exp(54.842763-6763.22/les['T']-4.210*np.log(les['T'])+
    0.000367*les['T']+np.tanh(0.0415*(les['T']-218.8))*
    (53.878-1331.22/les['T']-9.44523*np.log(les['T'])+0.014025*les['T'])) ) ) )
les['delta_aw'].attrs['units'] = '-'

In [ ]:
# plot LES environmental profiles

fig = plt.figure(figsize=[8,4])

ax = fig.add_subplot(231)
les["T"].plot(y='height')
ax.set_title('')

ax = fig.add_subplot(232)
les["ql"].plot(y='height')
ax.set_title('')

ax = fig.add_subplot(233)
les["RH"].plot(y='height')
ax.set_title('')

ax = fig.add_subplot(234)
les["delta_aw"].plot(y='height')
ax.set_title('')

ax = fig.add_subplot(235)
les["prec"].plot(y='height')
ax.set_title('')

ax = fig.add_subplot(236)
les["Ni"].plot(y='height')
ax.set_title('')

fig.tight_layout()

plt.show()

In [ ]:
# specify initial conditions and run parameters

# domain-uniform initial monomodal INP that are also hygroscopic

inp_init = 2.e0                       # particle number concentration [L-1]
inp_diam = 1.e-4                      # particle diameter [cm]
inp_surf = 4*np.pi*(inp_diam/2.)**2   # surface area per particle [cm2]

# specified ABIFM parameters
# illite
IN_c =  1.60671
IN_m = 14.96639
Jhet = np.double(10.**(IN_c + IN_m*les.delta_aw.data))    # nucleation rate [cm-2 s-1]

# specified quantities derived from LES offline (see Fridlind et al. 2012, doi:10.1175/JAS-D-11-052.1)

w_e_ent = 0.1e-3   # cloud-top entrainment rate [m/s]
tau_mix = 1800.    # boundary-layer mixing time scale [s]
v_f_ice = 0.3      # number-weighted ice crystal fall speed at the surface [m/s]

In [ ]:
# plot ABIFM nucleation rate profile

fig = plt.figure(figsize=[4,2])

ax1 = fig.add_subplot(111)
plt.plot(Jhet,les.height.data)
ax1.set_ylabel('Height [m]')
ax1.set_xlabel('J_het [cm-2 s-1]')

plt.show()

In [ ]:
# set up model grids and arrays
# prognostic variables: 
#     mod_inp(t,z) = INP number concentration [L-1]
#     mod_ni(t,z)  = ice crystal number concentration [L-1]
# reported process rates: 
#     mod_nuc(t,z) = INP nucleation rate = ice formation rate [L-1 s-1]

final_t = 6.*3600  # simulation time [s]
delta_t = 10.      # time step [s]

mod_nt  = int(final_t/delta_t)+1          # number of time steps
times   = np.arange(mod_nt)*delta_t       # time array [s]
mod_nz  = les.RH.data.size                # number of vertical grid cells
heights = les.height.data                     # height array [m]
delta_z = heights[mod_nz-1]-heights[mod_nz-2] # ∆z used in calculations, same for all levels [m]

mod_arr = xr.DataArray(np.zeros(shape=(mod_nt,mod_nz)),
                       coords=[('time', times),('height', heights)])
mod_arr.time.attrs['long_name']= 'Time'
mod_arr.time.attrs['units']= 's'
mod_arr.height.attrs['long_name']= 'Height'
mod_arr.height.attrs['units']= 'm'

In [ ]:
# run the model without separate interstitial INP class

# create empty DataArrays

mod_int = mod_arr.copy() # declared here only to zero any assigned values
mod_int.attrs['name'] = 'Interstitial INP'
mod_int.attrs['units'] = 'L-1'

mod_inp = mod_arr.copy()
mod_inp.attrs['name'] = 'INP'
mod_inp.attrs['units'] = 'L-1'

mod_ice = mod_arr.copy()
mod_ice.attrs['name'] = 'Ice'
mod_ice.attrs['units'] = 'L-1'

mod_nuc = mod_arr.copy()
mod_nuc.attrs['name'] = 'Nucleation Rate'
mod_nuc.attrs['units'] = 'L-1 s-1'

# initialize INP

mod_inp[dict(time=0)] = inp_init    # domain uniform monomodal INP (in- and above-cloud)

# loop over time steps

for it in range(1,mod_nt):
    # get values at last time step as 1D numpy arrays
    n_inp = mod_inp.isel(time=[it-1]).values.squeeze().copy()
    n_ice = mod_ice.isel(time=[it-1]).values.squeeze().copy()
    # nucleation of ice
    inp_activated = n_inp*Jhet*inp_surf*delta_t
    n_inp -= inp_activated
    n_ice += inp_activated
    # cloud top entrainment of INP
    inp_entrained = np.zeros(mod_nz)
    inp_entrained[mod_nz-1] = w_e_ent/delta_z*delta_t*(inp_init-n_inp[mod_nz-1])
    n_inp += inp_entrained
    # sedimentation of ice
    ice_sedimented_out = n_ice*v_f_ice/delta_z*delta_t
    ice_sedimented_in = np.zeros(mod_nz)
    ice_sedimented_in[0:mod_nz-2] = ice_sedimented_out[1:mod_nz-1]
    n_ice += ice_sedimented_in
    n_ice -= ice_sedimented_out
    # turbulent mixing
    ice_fully_mixed = np.zeros(mod_nz) + np.mean(n_ice)
    inp_fully_mixed = np.zeros(mod_nz) + np.mean(n_inp)
    ice_mixing = (ice_fully_mixed - n_ice)*delta_t/tau_mix
    inp_mixing = (inp_fully_mixed - n_inp)*delta_t/tau_mix
    n_ice += ice_mixing
    n_inp += inp_mixing
    # save new values and nucleation rate
    mod_ice[dict(time=it)] = n_ice.copy()
    mod_inp[dict(time=it)] = n_inp.copy()
    mod_nuc[dict(time=it)] = inp_activated.copy()

# estimate precipitation rate as a multiple of ice number concentration

mod_pcb = mod_ice.copy()*Pcb_per_Ni

In [ ]:
# run the model accounting for entrainment to interstitial class

# create empty DataArrays

mod_int = mod_arr.copy()
mod_int.attrs['name'] = 'Interstitial INP'
mod_int.attrs['units'] = 'L-1'

mod_inp = mod_arr.copy()
mod_inp.attrs['name'] = 'Incorporated INP'
mod_inp.attrs['units'] = 'L-1'

mod_ice = mod_arr.copy()
mod_ice.attrs['name'] = 'Ice'
mod_ice.attrs['units'] = 'L-1'

mod_nuc = mod_arr.copy()
mod_nuc.attrs['name'] = 'Nucleation Rate'
mod_nuc.attrs['units'] = 'L-1 s-1'

# initialize interstitial INP uniformly to the default array (in- and above-cloud)

mod_int[dict(time=0)] = inp_init    # domain uniform monomodal INP (in- and above-cloud)

# loop over time steps

for it in range(1,mod_nt):
    # get values at last time step as 1D numpy arrays
    n_int = mod_int.isel(time=[it-1]).values.squeeze().copy()
    n_inp = mod_inp.isel(time=[it-1]).values.squeeze().copy()
    n_ice = mod_ice.isel(time=[it-1]).values.squeeze().copy()
    # incorporation of interstitial INP
    inp_incorporated = np.zeros(mod_nz)
    below_cloud = np.where(les.ql.data<1.e-3)
    inp_incorporated[below_cloud] = n_int[below_cloud]
    n_int -= inp_incorporated
    n_inp += inp_incorporated
    # nucleation of ice from both interstitial and incorporated INP with ABIFM
    int_activated = n_int*Jhet*inp_surf*delta_t
    n_int -= int_activated
    inp_activated = n_inp*Jhet*inp_surf*delta_t
    n_inp -= inp_activated
    n_ice += int_activated + inp_activated
    # cloud top entrainment of INP (proportional to INP aloft - Ninp+Nint at cloud top)
    inp_entrained = np.zeros(mod_nz)
    inp_entrained[mod_nz-1] = w_e_ent/delta_z*delta_t*(inp_init-(n_int[mod_nz-1]+n_inp[mod_nz-1]))
    n_int += inp_entrained
    # sedimentation of ice
    ice_sedimented_out = n_ice*v_f_ice/delta_z*delta_t
    ice_sedimented_in = np.zeros(mod_nz)
    ice_sedimented_in[0:mod_nz-2] = ice_sedimented_out[1:mod_nz-1]
    n_ice -= ice_sedimented_out
    n_ice += ice_sedimented_in
    # turbulent mixing
    ice_fully_mixed = np.zeros(mod_nz) + np.mean(n_ice)
    inp_fully_mixed = np.zeros(mod_nz) + np.mean(n_inp)
    int_fully_mixed = np.zeros(mod_nz) + np.mean(n_int)
    ice_mixing = (ice_fully_mixed - n_ice)*delta_t/tau_mix
    inp_mixing = (inp_fully_mixed - n_inp)*delta_t/tau_mix
    int_mixing = (int_fully_mixed - n_int)*delta_t/tau_mix
    n_ice += ice_mixing
    n_inp += inp_mixing
    n_int += int_mixing
    # save new values and nucleation rate
    mod_ice[dict(time=it)] = n_ice.copy()
    mod_inp[dict(time=it)] = n_inp.copy()
    mod_int[dict(time=it)] = n_int.copy()
    mod_nuc[dict(time=it)] = int_activated + inp_activated
    
# estimate precipitation rate as a multiple of ice number concentration

mod_pre = mod_ice.copy()*Pcb_per_Ni

In [ ]:
# plot time series of domain-mean quantities

fig = plt.figure(figsize=[8,6])

ax = fig.add_subplot(321)
mod_inp.mean(dim='height').plot()
ax.set_ylabel('Incorporated INP [L-1]')

if (mod_int.max()>0):
    ax = fig.add_subplot(322)
    mod_int.mean(dim='height').plot()
    ax.set_ylabel('Interstitial INP [L-1]')

ax = fig.add_subplot(323)
mod_ice.mean(dim='height').plot()
ax.set_ylabel('Mean Ice [L-1]')

ax = fig.add_subplot(324)
mod_nuc.mean(dim='height').plot()
ax.set_ylabel('Nucleation Rate [L-1 s-1]')

ax = fig.add_subplot(325)
mod_pcb[dict(height=izt_cloud_base)].plot()
ax.set_ylabel('Cloud Base Prec [mm h-1]')

fig.tight_layout()

plt.show()

In [ ]:
# plot time-height contours

fig = plt.figure(figsize=[8,4])

ax = fig.add_subplot(221)
mod_inp.plot.contourf('time')
ax.set_title('Incorporated INP')

if (mod_int.max()>0):
    ax = fig.add_subplot(222)
    mod_int.plot.contourf('time')
    ax.set_title('Interstitial INP')

ax = fig.add_subplot(223)
mod_nuc.plot.contourf('time')
ax.set_title('Nucleation Rate')

ax = fig.add_subplot(224)
mod_ice.plot.contourf('time')
ax.set_title('Ice')

fig.tight_layout()

plt.show()

In [ ]:
# plot profiles at a fixed time [s]

my_time = final_t

fig = plt.figure(figsize=[8,4])

ax = fig.add_subplot(221)
mod_inp.sel(time=my_time,method='nearest').plot(y='height')
ax.set_xlabel('Incorporated INP [L-1]')

if (mod_int.max()>0):
    ax = fig.add_subplot(222)
    mod_int.sel(time=my_time,method='nearest').plot(y='height')
    ax.set_xlabel('Interstitial INP [L-1]')

ax = fig.add_subplot(223)
mod_nuc.sel(time=my_time,method='nearest').plot(y='height')
ax.set_xlabel('Nucleation Rate [L-1 s-1]')
plt.xscale('log')

ax = fig.add_subplot(224)
mod_ice.sel(time=my_time,method='nearest').plot(y='height')
ax.set_xlabel('Ice [L-1]')

fig.tight_layout()

plt.show()